In [5]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from statistics import mean
from datetime import date

In [6]:
# start_date = '2021-1-1' # start of previous cycle
# start_date = '2022-1-1' # start of downtrend
start_date = '2020-1-1' # start of uptrend
# start_date = '2023-4-10' # start of MAGS history
# start_date = '2024-2-28' # start of MAGX history
weeks = 52
rows = []
for symbol in [
    'SPY','QQQ','IGM','XLKS.MI','SMH','USD','TQQQ','FNGU','FNGS','MAGS',
    'MAGX','MAG7.L'
]:
    tickerData = yf.Ticker(symbol)
    history = tickerData.history(interval='1wk', start=start_date, end=date.today()) # prices are adjusted for splits
    changes = history.Close.pct_change(periods=weeks)
    rows.append({
        'symbol': symbol,
        'weeks': len(history),
        'change': changes.mean(),
        'std': changes.std(),
        'score': changes.mean() / changes.std(),
    })

df = pd.DataFrame(rows).sort_values(by='score', ascending=False)
display(df)
df.to_csv('data/stocks-52w.csv', index=False)


,symbol,weeks,change,std,score
9,MAGS,86,0.518057,0.081659,6.344160
3,XLKS.MI,257,0.273342,0.222073,1.230868
4,SMH,257,0.373083,0.375951,0.992371
0,SPY,257,0.169670,0.185855,0.912913
5,USD,257,0.946128,1.036571,0.912748
1,QQQ,257,0.205380,0.262003,0.783883
8,FNGS,257,0.328178,0.436482,0.751872
2,IGM,257,0.225046,0.308464,0.729570
7,FNGU,257,1.185751,1.751757,0.676892
6,TQQQ,257,0.586532,0.880773,0.665929
